### This notebook aims to show what the benchmark.jl and benchmark_glaciers.jl scripts do 

In [1]:
#Useful packages
include("oggm_access.jl")
include("1D_SIA.jl")
include("1D_SIA_raw.jl")
using NCDatasets
using BenchmarkTools
using Distributed

2023-05-12 11:18:15: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-05-12 11:18:15: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-05-12 11:18:15: oggm.cfg: Multiprocessing: using all available processors (N=16)
2023-05-12 11:18:15: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2023-05-12 11:18:15: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2023-05-12 11:18:15: oggm.cfg: PARAMS['store_fl_diagnostics'] changed from `False` to `True`.
2023-05-12 11:18:15: oggm.cfg: Multiprocessing switched ON after user settings.
2023-05-12 11:18:35: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-05-12 11:18:35: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-05-12 11:18:35: oggm.cfg: Multiprocessing: using all available processors (N=16)
2023-05-12 11:18:35: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
202

### Benchmarking different solvers

In [2]:
#Here definition of the function for the pmap

@everywhere begin
using DifferentialEquations

using JLD2
using Random
using SugarBLAS
using Statistics: median
# using AbbreviatedStackTraces
using Logging: global_logger
using Revise, BenchmarkTools
using LinearAlgebra
using TerminalLoggers: TerminalLogger
using DataFrames
global_logger(TerminalLogger())

function benchmark_setting(setting)

    ude_benchmark = Dict("ude_settings"=>[], "time_stats"=>[])

    UDE_settings = Dict("reltol"=>reltol,"solver"=>[])
    UDE_settings["solver"] = setting

    println("Benchmarking UDE settings: ", UDE_settings)
    push!(ude_benchmark["ude_settings"], UDE_settings)

    try
        t_stats = @timed glacier_evolution(gdir=gdir, 
        dx=dx_o, # grid resolution in m
        nx=length(bed_o),  # grid size
        width=widths_o,  # glacier width in m 
        glen_a= 2.4e-24,  # ice stiffness 2.4e-24
        n_years=15.0,  # simulation time in years
        solver = setting,
        reltol=UDE_settings["reltol"],
        bed_hs=bed_o,
        surface_ini=surface_o)

        # Save stats for each solver

        push!(ude_benchmark["time_stats"], t_stats)

  
    catch error
        println("ERROR: ", error)
        @warn "Solver not working. Skipping..."
    end

    GC.gc()
    return ude_benchmark
end


end #everywhere 

In [3]:
### Main ###


#Choose one glacier
rgi_ids=["RGI60-11.01450"]
gdirs=init_gdirs(rgi_ids)
gdir=gdirs[1]


#Getting all needed parameters 
PARAMS["evolution_model"] = "FluxBased"
tasks.init_present_time_glacier(gdir)

fls=gdir.read_pickle("model_flowlines")
bed_o = fls[end].bed_h
surface_o = fls[end].surface_h
widths_o = fls[end].widths_m
dx_o = fls[end].dx_meter

diag = gdir.get_diagnostics()
glen_a_o = diag["inversion_glen_a"]


#Solvers
reltol = 1e-6

ude_solvers = [Euler(),BS3(),OwrenZen3(), Ralston(), RDPK3Sp35(), CKLLSRK54_3C()]
    
#Tried following solvers but glacier exceeds boundaries : 
#VCABM(), Vern6(), AN5(),AB3(), KenCarp3(autodiff=false),TRBDF2(autodiff=false),ROCK4(),QNDF(autodiff=false), Tsit5(),Rodas4P()
#ImplicitEuler, ImplicitMidpoint,RK4, DP5

#Vern8,9 , Feagin10,MSRK5, KuttaPRK2p5,SSPRK22, TanYam7 instables

ude_benchmarks=zeros(length(ude_solvers))

# Benchmark every solver in parallel
ude_benchmarks = pmap(ude_solver -> benchmark_setting(ude_solver), ude_solvers) 
                
save_object("data/benchmark.jld2",ude_benchmarks)


2023-05-12 11:19:16: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:19:17: oggm.cfg: PARAMS['evolution_model'] changed from `SemiImplicit` to `FluxBased`.


Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => Euler())
Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => BS3(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)))
Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => OwrenZen3(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)))
Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => Ralston(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)))
Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => RDPK3Sp35(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)))
Benchmarking UDE settings: Dict{String, Any}("reltol" => 1.0e-6, "solver" => CKLLSRK54_3C(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)))


In [5]:
### A look at the results ###
ude = load("data/benchmark.jld2")

n=length(ude_solvers)
stats = DataFrame(:id => 1:n, :name => ude_solvers, 
                  :reltol => ones(n)*reltol,
                  :t => [ude["single_stored_object"][i]["time_stats"][1].time for i =1:n],
                  :bytes => [ude["single_stored_object"][i]["time_stats"][1].bytes for i =1:n],
                  :gctime => [ude["single_stored_object"][i]["time_stats"][1].gctime for i =1:n])

show(stats,summary=false)

┌ Warning: type Main.#get_mb2#15{PyObject} does not exist in workspace;
│ reconstructing
└ @ JLD2 /home/gimenelu/.julia/packages/JLD2/ryhNR/src/data/reconstructing_datatypes.jl:495
┌ Warning: type
│ Main.#iceflow!#16{JLD2.ReconstructedTypes.var"##Main.#get_mb2#15{PyObject}#477",Vector{Float64},Vector{Float64},Vector{Float64},Vector{Float64}}
│ does not exist in workspace; reconstructing
└ @ JLD2 /home/gimenelu/.julia/packages/JLD2/ryhNR/src/data/reconstructing_datatypes.jl:495


 Row │ id     name                               reltol   t         bytes      ⋯
     │ Int64  Ordinary…                          Float64  Float64   Int64      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │     1  Euler()                             1.0e-6  0.767518  143225190  ⋯
   2 │     2  BS3(stage_limiter! = trivial_lim…   1.0e-6  0.30291    25126765
   3 │     3  OwrenZen3(stage_limiter! = trivi…   1.0e-6  0.283958   24232000
   4 │     4  Ralston(stage_limiter! = trivial…   1.0e-6  0.288019   24875632
   5 │     5  RDPK3Sp35(stage_limiter! = trivi…   1.0e-6  0.411093   37914296  ⋯
   6 │     6  CKLLSRK54_3C(stage_limiter! = tr…   1.0e-6  0.229528   18852064
                                                                1 column omitted

### Benchmarking different glaciers

In [13]:
#Definition of the function to pass in pmap
function benchmark_setting_gla(rgi)
    rgi_id = [rgi]
    gdirs=init_gdirs(rgi_id)
    gdir=gdirs[1]
    gla_name=gdir.name
    gla_nb = findall(x->x==rgi,rgi_ids)[1]

    #Getting the flowlines 
    tasks.init_present_time_glacier(gdir)

    fls=gdir.read_pickle("model_flowlines")
    bed_o = fls[end].bed_h
    surface_o = fls[end].surface_h
    widths_o = fls[end].widths_m
    dx_o = fls[end].dx_meter

    diag = gdir.get_diagnostics()
    glen_a_o = diag["inversion_glen_a"]



    ude_benchmark = Dict("glacier_id"=>[], "time_stats"=>[],"time_stats_oggm"=>[])

    println("Benchmarking glacier settings: ", rgi)
    push!(ude_benchmark["glacier_id"], rgi)

    try
        t_stats = @timed glacier_evolution(gdir=gdir, 
        dx=dx_o, # grid resolution in m
        nx=length(bed_o),  # grid size
        width=widths_o,  # glacier width  in years
        glen_a= glen_a_o,
        n_years=15.0,
        solver = Ralston(),
        reltol=1e-6,
        bed_hs=bed_o,
        surface_ini=surface_o)

        push!(ude_benchmark["time_stats"], t_stats)

        t_stats_o =@timed workflow.execute_entity_task(tasks.run_from_climate_data, gdir,
                                climate_filename="climate_historical",
                                ys=2004, ye=2019,store_fl_diagnostics=true)
            
        push!(ude_benchmark["time_stats_oggm"], t_stats_o)
        


    catch error
        println("ERROR: ", error)
        @warn "Solver not working. Skipping..."
    end

    #=
    iceflow_sol = glacier_evolution(gdir=gdir, dx=dx_o, nx=length(bed_o), width=widths_o,glen_a= 2.4e-24,n_years=15.0,solver = Ralston(),
    reltol=1e-6,bed_hs=bed_o,surface_ini=surface_o)

    plot(bed_o, c="brown",label="bed",title="$gla_name",ylabel="Elevation (m.a.s.l.)")
    plot!(iceflow_sol[end] .+ bed_o, c="blue",label="surface solver")
    plot!(surface_o,color="green",label="surface initiale")

    f = gdir.get_filepath("fl_diagnostics")
    ds = NCDataset(f)
    fl_id=0
    ds2=ds.group["fl_$fl_id"]
    plot!(ds2["bed_h"][:,end]+ds2["thickness_m"][:,end],linestyle=:dash,color="black",label="surface oggm")

    savefig("myplot_$gla_nb.png")   
    =#
    
    
    GC.gc()
    return ude_benchmark
end



benchmark_setting_gla (generic function with 1 method)

In [14]:
### Main ###

rgi_ids=["RGI60-11.03638","RGI60-11.03671","RGI60-11.03643","RGI60-11.03674","RGI60-11.03756", #Argentière, Gébroulaz, Mer de Glace,St-Sorlin, Sarennes
        "RGI60-16.00543","RGI60-16.01339", #Zongo, Antizana
        "RGI60-11.03232", #Ossoue
        "RGI60-15.03591"] #Mera 

ude_benchmarks=zeros(length(rgi_ids))

# Benchmark every solver in parallel
ude_benchmarks = pmap(glacier -> benchmark_setting_gla(glacier), rgi_ids) 
                
save_object("data/benchmark_gla_glen_a.jld2",ude_benchmarks)


Benchmarking glacier settings: RGI60-11.03638


2023-05-12 11:30:38: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:30:39: oggm.workflow: Execute entity tasks [run_from_climate_data] on 1 glaciers


Benchmarking glacier settings: RGI60-11.03671


2023-05-12 11:30:40: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:30:40: oggm.workflow: Execute entity tasks [run_from_climate_data] on 1 glaciers


Benchmarking glacier settings: RGI60-11.03643


...

2023-05-12 11:30:45: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:30:48: oggm.workflow: Execute entity tasks [run_from_climate_data] on 1 glaciers


Benchmarking glacier settings: RGI60-11.03232


2023-05-12 11:30:48: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:30:49: oggm.workflow: Execute entity tasks [run_from_climate_data] on 1 glaciers
2023-05-12 11:30:49: oggm.core.flowline: You are attempting to run_with_climate_data at dates prior to the RGI inventory date. This may indicate some problem in your workflow. Consider using `fixed_geometry_spinup_yr` for example.


Benchmarking glacier settings: RGI60-15.03591


2023-05-12 11:30:49: oggm.workflow: Execute entity tasks [GlacierDirectory] on 1 glaciers
2023-05-12 11:30:50: oggm.workflow: Execute entity tasks [run_from_climate_data] on 1 glaciers


In [12]:
### A look at the results ###
ude = load("data/benchmark_gla.jld2")

n=length(rgi_ids)
stats_gla = DataFrame(:id => 1:n, :name => rgi_ids, 
                  :t => [ude["single_stored_object"][i]["time_stats"][1].time for i =1:n],
                  :bytes => [ude["single_stored_object"][i]["time_stats"][1].bytes for i =1:n],
                  :t_oggm=> [ude["single_stored_object"][i]["time_stats_oggm"][1].time for i =1:n],
                  :bytes_oggm => [ude["single_stored_object"][i]["time_stats_oggm"][1].bytes for i =1:n])

show(stats_gla,summary=false)

 Row │ id     name            t          bytes      t_oggm     bytes_oggm 
     │ Int64  String          Float64    Int64      Float64    Int64      
─────┼────────────────────────────────────────────────────────────────────
   1 │     1  RGI60-11.03638  0.336881    24044256  0.409703     16820684
   2 │     2  RGI60-11.03671  0.119851     9050032  0.0702186        3600
   3 │     3  RGI60-11.03643  0.288596    22733296  0.180085         3600
   4 │     4  RGI60-11.03674  0.0755701    6757952  0.0528422        3600
   5 │     5  RGI60-11.03756  0.0935065    5541136  0.0669694        3600
   6 │     6  RGI60-16.00543  0.201645    15302208  0.0862814        3600
   7 │     7  RGI60-16.01339  2.50455    157360416  0.22295          3600
   8 │     8  RGI60-11.03232  0.134972     8754320  0.0737252        3600
   9 │     9  RGI60-15.03591  0.212765    14337840  0.0883358        3600

In [15]:
### If using the calibrated creep parameter A

### A look at the results ###
ude = load("benchmark_gla_glen_a.jld2")

n=length(rgi_ids)
stats_gla_glen_a = DataFrame(:id => 1:n, :name => rgi_ids, 
                  :t => [ude["single_stored_object"][i]["time_stats"][1].time for i =1:n],
                  :bytes => [ude["single_stored_object"][i]["time_stats"][1].bytes for i =1:n],
                  :t_oggm=> [ude["single_stored_object"][i]["time_stats_oggm"][1].time for i =1:n],
                  :bytes_oggm => [ude["single_stored_object"][i]["time_stats_oggm"][1].bytes for i =1:n])

show(stats_gla_glen_a,summary=false)

 Row │ id     name            t          bytes      t_oggm     bytes_oggm 
     │ Int64  String          Float64    Int64      Float64    Int64      
─────┼────────────────────────────────────────────────────────────────────
   1 │     1  RGI60-11.03638  0.862418    82024416  0.126811         3600
   2 │     2  RGI60-11.03671  0.141137    12342704  0.141006         3600
   3 │     3  RGI60-11.03643  1.06063    102968176  0.159212         3600
   4 │     4  RGI60-11.03674  0.154855    11106368  0.0626282        3600
   5 │     5  RGI60-11.03756  0.0811473    5906192  0.0586346        3600
   6 │     6  RGI60-16.00543  0.375087    34792848  0.159814         3600
   7 │     7  RGI60-16.01339  2.36982    204885216  0.161121         3600
   8 │     8  RGI60-11.03232  0.165031    13946000  0.055719         3600
   9 │     9  RGI60-15.03591  0.295367    27006256  0.0736391        3600